In [8]:
from elasticsearch import Elasticsearch, helpers
import os
import re
import json
import logging
import operator

In [9]:
def connect_elasticsearch():
    _es = None
    _es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if _es.ping():
        print('Yay Connected')
    else:
        print('Awww it could not connect!')
    return _es
if __name__ == '__main__':
  logging.basicConfig(level=logging.ERROR)

In [10]:
es = connect_elasticsearch()

Yay Connected


## Load Query File 

In [11]:
query_file_path = "D:\\CS 6200\\AP_DATA\\query_desc.51-100.short.txt"
queries = dict()
with open(query_file_path) as f:
    for line in f:
        if len(line.strip()) > 0:
            query_no = re.search(r'\d+', line).group()
            line = line.lstrip('0123456789.- ')
            queries[query_no] = line.rstrip()

In [12]:
query_relevant_terms = dict()

In [13]:
for key in queries:
    relevant_terms = dict()
    analyzerTokens = es.indices.analyze(index = "assignment1", body = {"tokenizer" : "standard", "filter" : ["lowercase", "my_stemmer"], "text" : queries[key]})

    query_words = []

    for token in analyzerTokens["tokens"]:
        query_words.append(token["token"])

    for word in query_words:

        statistics = es.search(index = "assignment1", body={"query" : {"terms" : {"text" : [word]}},
    "aggregations" : {"significantCrimeTypes" : {"significant_terms" : {"field" : "text"}}},"size": 0})
        
        if len(statistics["aggregations"]["significantCrimeTypes"]["buckets"])>0:
            for term in statistics["aggregations"]["significantCrimeTypes"]["buckets"]:
                if term["key"] not in relevant_terms.keys():
                    relevant_terms[term["key"]] = 1
                else:
                    relevant_terms[term["key"]] += 1
    sorted_d = dict( sorted(relevant_terms.items(), key=operator.itemgetter(1),reverse=True))
    sortedKeys = list(sorted_d)

    query_relevant_terms[key] = sortedKeys[:2]   
print(query_relevant_terms)       


{'85': ['corrupt', 'bribe'], '59': ['thunderstorm', 'storm'], '56': ['prime', 'minist'], '71': ['troop', 'soldier'], '64': ['parti', 'leader'], '62': ['militari', 'coup'], '93': ['nation', 'presid'], '99': ['iran', 'contra'], '58': ['rail', 'railroad'], '77': ['poach', 'poacher'], '54': ['orbit', 'nasa'], '87': ['alleg', 'case'], '94': ['crime', 'convict'], '100': ['high', 'regul'], '89': ['invest', 'investor'], '61': ['iran', 'contra'], '95': ['comput', 'softwar'], '68': ['employe', 'worker'], '57': ['mci', "mci'"], '97': ['fiber', 'optic'], '98': ['fiber', 'optic'], '60': ['pai', 'salari'], '80': ['presidenti', 'candid'], '63': ['machin', 'chip'], '91': ['scientist', 'militari']}


In [14]:
outF = open("D:\\CS 6200\\AP_DATA\\queries_pr2.txt", "a")
for key in queries:
    string = ""
    for i in range(0, len(query_relevant_terms[key])):
        string += " "+query_relevant_terms[key][i]
    modified_query = queries[key] +string
    line = str(key) + ". " + modified_query + "\n"
    outF.write(line)
outF.close()